In [ ]:
# @title Step 1 { vertical-output: true }
import shutil
shutil.copytree('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/training_data/conv_lstm/', '/content/training_data_conv_lstm/', dirs_exist_ok=True)
shutil.copytree('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/training_data/Dendritic_CapsNets/', '/content/training_data_Dendritic_CapsNets/', dirs_exist_ok=True)
shutil.copytree('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/training_data/cnn/', '/content/training_data_cnn/', dirs_exist_ok=True)
# Install necessary libraries
!pip install scikit-learn
!pip install joblib soundfile speechpy librosa matplotlib
!pip install joblib==1.2.0
!pip install np_utils
!pip install lime
!pip install shap

import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
#from keras.utils import np_utils
import joblib
import os

#clstm_model = tf.keras.models.load_model("/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/conv_lstm_classifier.tf")
#clstm_model.summary()
#capsnet_model = tf.keras.models.load_model("/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/Dendritic_CapsNets.tf")
#capsnet_model.summary()

In [ ]:
# @title CNN LIME - Random Data - Working
# Import necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

# Create a simple CNN model
def create_cnn_model(input_shape=(64, 64, 3), num_classes=2):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Generate sample data
def generate_sample_data(num_samples=1000, img_size=(64, 64, 3), num_classes=2):
    X = np.random.rand(num_samples, *img_size)
    y = np.random.randint(num_classes, size=num_samples)
    return X, y

# Split data into training and testing sets
X, y = generate_sample_data()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode the labels
num_classes = len(np.unique(y))
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.eye(num_classes)[y_test]

# Train the CNN model
input_shape = x_train.shape[1:]
cnn_test_model = create_cnn_model(input_shape, num_classes)
cnn_test_model.fit(x_train, y_train_one_hot, epochs=4, batch_size=32, validation_split=0.2)
cnn_test_model.summary()
# Now, let's use LIME for explainability on a single test instance
# Pick a random test instance
print("x_test - Shape")
print(x_test.shape)

test_instance = x_test[0]

# Create a LIME explainer for image classification
explainer = lime_image.LimeImageExplainer()

# Explain the model's prediction on the test instance
explanation = explainer.explain_instance(test_instance, cnn_test_model.predict, top_labels=1, hide_color=0, num_samples=1000)

# Display the original image and the LIME explanation
plt.subplot(1, 2, 1)
plt.imshow(test_instance)
plt.title('Original Image')

plt.subplot(1, 2, 2)
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.title('LIME Explanation')

plt.show()




In [51]:
# @title CNN LIME - Random Audio Data - NOT Working
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from lime.lime_image import LimeImageExplainer
import shap

# Generate synthetic data
num_samples = 200
num_features = 1047
data_shape = (num_samples, num_features, 40, 1)
data = np.random.normal(loc=0.0, scale=1.0, size=data_shape)

# Generate random binary labels (0 or 1)
labels = np.random.randint(2, size=num_samples)

# Split data into train and test sets
train_size = 0.8
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, train_size=train_size, random_state=200
)

# Create a simple CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(num_features, 40, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=2, batch_size=32)

# LIME explanation
explainer = LimeImageExplainer()
explanation = explainer.explain_instance(test_data[0], model.predict)
explanation.show_in_notebook()

# SHAP explanation
#explainer_shap = shap.Explainer(model, train_data)
#shap_values = explainer_shap(test_data)
#shap.summary_plot(shap_values, test_data)

# Now you have both LIME and SHAP explanations!









Epoch 1/2
5/5 [==============================] - 12s 2s/step - loss: 36.4196 - accuracy: 0.5625
Epoch 2/2
5/5 [==============================] - 7s 1s/step - loss: 30.9415 - accuracy: 0.4437


ValueError: the input array must have size 3 along `channel_axis`, got (1047, 40, 1)

In [ ]:
# @title CNN LIME - Random Audio Data - Working
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from lime import lime_image
import matplotlib.pyplot as plt

###################################################
def load_data(load_from_file,name):
    if load_from_file:
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = '/content/training_data_cnn'

        x_train = joblib.load(os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        x_test = joblib.load(os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        y_train = joblib.load(os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        y_test = joblib.load(os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        num_labels = joblib.load(os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #data = joblib.load(os.path.join(save_dir, 'data_'+name+'.joblib'))
        #labels = joblib.load(os.path.join(save_dir, 'labels_'+name+'.joblib'))
    else:
        #x_train, x_test, y_train, y_test, num_labels = extract_data()
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = current_dir + subdir
        joblib.dump(x_train, os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        joblib.dump(x_test, os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        joblib.dump(y_train, os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        joblib.dump(y_test, os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        joblib.dump(num_labels, os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #joblib.dump(data, os.path.join(save_dir, 'data_'+name+'.joblib'))
        #joblib.dump(labels, os.path.join(save_dir, 'labels_'+name+'.joblib'))
    return x_train,x_test,y_train,y_test,num_labels#,data, labels
###################################################

loadfromfile=True
x_train,x_test,y_train,y_test,num_labels =load_data(load_from_file=loadfromfile,name= 'cnn')

from keras import utils
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
in_shape = x_train[0].shape

x_train = x_train.reshape(x_train.shape[0], in_shape[1], in_shape[0], 1)
x_test = x_test.reshape(x_test.shape[0], in_shape[1], in_shape[0], 1)


num_timesteps, num_features = x_train.shape[1], x_train.shape[2]
#num_classes = y_train.shape[1]
num_classes=8
###################################################

# Assuming x_test has shape (200, 1047, 40, 1)
num_samples, duration, n_mels, channels = x_test.shape

# Reshape data for the LimeImageExplainer
x_test_reshaped = x_test.reshape((-1, n_mels, duration, channels))

# Create a simple CNN model for audio classification
def create_audio_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a LIME explainer for image classification
explainer = lime_image.LimeImageExplainer()

# Explain the model's predictions on multiple test instances
for i in range(num_samples):
    test_instance = x_test_reshaped[i]

    # Train the CNN model
    audio_cnn_model = create_audio_cnn_model(test_instance.shape)
    audio_cnn_model.fit(x_train, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.2)

    # Explain the model's prediction on the test instance
    explanation = explainer.explain_instance(test_instance, audio_cnn_model.predict, top_labels=1, hide_color=0, num_samples=1000)

    # Display the original spectrogram and the LIME explanation
    plt.subplot(1, 2, 1)
    plt.imshow(test_instance[:, :, 0], cmap='viridis', origin='lower')
    plt.title('Original Spectrogram')

    plt.subplot(1, 2, 2)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
    plt.imshow(temp, cmap='viridis', origin='lower')
    plt.title('LIME Explanation')

    plt.show()




In [ ]:
# @title DendCaps LIME
#from include.functions import initialise, load_data
#from include.stats import plot_accuracy_loss
import numpy as np
import sys
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import sys
from keras import Sequential
from keras.layers import LSTM,Dense, Dropout, Conv2D, Flatten,BatchNormalization, Activation, MaxPooling2D,Reshape
import joblib
import os
###################################################
def load_data(load_from_file,name):
    if load_from_file:
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = '/content/training_data_Dendritic_CapsNets'

        x_train = joblib.load(os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        x_test = joblib.load(os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        y_train = joblib.load(os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        y_test = joblib.load(os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        num_labels = joblib.load(os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #data = joblib.load(os.path.join(save_dir, 'data_'+name+'.joblib'))
        #labels = joblib.load(os.path.join(save_dir, 'labels_'+name+'.joblib'))
    else:
        #x_train, x_test, y_train, y_test, num_labels = extract_data()
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = current_dir + subdir
        joblib.dump(x_train, os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        joblib.dump(x_test, os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        joblib.dump(y_train, os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        joblib.dump(y_test, os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        joblib.dump(num_labels, os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #joblib.dump(data, os.path.join(save_dir, 'data_'+name+'.joblib'))
        #joblib.dump(labels, os.path.join(save_dir, 'labels_'+name+'.joblib'))
    return x_train,x_test,y_train,y_test,num_labels#,data, labels
###################################################
###################################################
# Squash function
def squash(x, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(x), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + 1e-7)
    return scale * x

###################################################
# Define Capsule Layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(name='capsule_kernel',
                                      shape=(input_shape[-1], self.num_capsules * self.dim_capsule),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs):
        # Following is a simplified routing algorithm
        inputs = tf.keras.backend.reshape(inputs, (-1, inputs.shape[-1]))
        u = tf.keras.backend.dot(inputs, self.kernel)
        u = tf.keras.backend.reshape(u, (-1, self.num_capsules, self.dim_capsule))
        b = tf.zeros_like(u[:, :, 0])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            s = tf.reduce_sum(c[:, :, tf.newaxis] * u, axis=1)
            v = squash(s)
            b = b + tf.reduce_sum(u * v[:, tf.newaxis, :], axis=2)
        return v

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_capsules': self.num_capsules,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        })
        return config
###################################################
# Custom Dendritic Layer
class DendriticLayer(layers.Layer):
    def __init__(self, units, segments):
        super(DendriticLayer, self).__init__()
        self.units = units
        self.segments = segments

    def build(self, input_shape):
        self.segment_weights = []
        self.segment_biases = []
        for _ in range(self.segments):
            self.segment_weights.append(self.add_weight(
                name='dend1',
                shape=(input_shape[-1], self.units),
                initializer='random_normal',
                trainable=True))
            self.segment_biases.append(self.add_weight(
                name='dend2',
                shape=(self.units,),
                initializer='zeros',
                trainable=True))

        self.soma_weight = self.add_weight(
            name='dend3',
            shape=(self.units * self.segments, self.units),
            initializer='random_normal',
            trainable=True)
        self.soma_bias = self.add_weight(
            name='dend4',
            shape=(self.units,),
            initializer='zeros',
            trainable=True)

    def call(self, inputs):
        outputs = []
        for i in range(self.segments):
            x = tf.matmul(inputs, self.segment_weights[i]) + self.segment_biases[i]
            x = tf.nn.relu(x)
            outputs.append(x)

        concat = tf.concat(outputs, axis=-1)
        soma_output = tf.matmul(concat, self.soma_weight) + self.soma_bias
        soma_output = tf.nn.relu(soma_output)
        return soma_output

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'segments': self.segments
        })
        return config
###################################################
def DendCaps():

    loadfromfile=True
    x_train,x_test,y_train,y_test,num_labels =load_data(load_from_file=loadfromfile,name= 'Dendritic_CapsNets')

    from keras import utils
    y_train = utils.to_categorical(y_train)
    y_test = utils.to_categorical(y_test)
    in_shape = x_train[0].shape

    x_train = x_train.reshape(x_train.shape[0], in_shape[1], in_shape[0], 1)
    x_test = x_test.reshape(x_test.shape[0], in_shape[1], in_shape[0], 1)

    num_timesteps, num_features = x_train.shape[1], x_train.shape[2]
    num_classes = 8   #y1_train.shape[1]
    ###################################################
    ###################################################
    # Define CNN with Dendritic Layer Followed by Capsule Layer
    import matplotlib.pyplot as plt

    input_shape = (num_timesteps, num_features, 1)
    inputs = tf.keras.Input(shape=input_shape)
    conv1 = layers.Conv2D(128, (3, 3), activation='relu')(inputs)
    conv2 = layers.Conv2D(32, (3, 3), activation='relu')(conv1)
    pooling1 = layers.MaxPooling2D(pool_size=2)(conv2)
    dropout1 = layers.Dropout(0.5)(pooling1)
    flatten1 = layers.Flatten()(dropout1)
    dendritic = DendriticLayer(64, 2)(flatten1)  # 64 units, 2 segments
    capsule = CapsuleLayer(num_capsules=10, dim_capsule=32, routings=3)(dendritic)
    #flatten2 = layers.Flatten()(capsule)
    dense1 = Dense(32, activation='relu')(capsule)
    dropout2 = layers.Dropout(0.5)(dense1)
    outputs = Dense(num_classes, activation='softmax')(dropout2)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile and Train
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
  #####################
    loss_history = []
    accuracy_history = []

    epochs = 100

    for epoch in range(epochs):
        # Train the model for one epoch
        history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test,y_test), verbose=0)

        # Record loss and accuracy values for plotting
        loss_history.append(history.history['loss'][0])
        accuracy_history.append(history.history['accuracy'][0])

        # Display the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss_history[-1]:.4f} - Accuracy: {accuracy_history[-1]:.4f}")

        # Plot loss and accuracy
        """
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(loss_history)
        plt.title('Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 2, 2)
        plt.plot(accuracy_history)
        plt.title('Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.tight_layout()
        plt.show()
        """

  #####################
    history = model.fit(x_train, y_train, batch_size=32, epochs=5,shuffle=True,validation_split=0.05)

    # Pick a random test instance
    test_instance = x_test[0]

    # Create a LIME explainer for image classification
    explainer = lime_image.LimeImageExplainer()

    # Explain the model's prediction on the test instance
    explanation = explainer.explain_instance(test_instance, cnn_model.predict, top_labels=1, hide_color=0, num_samples=1000)

    # Display the original image and the LIME explanation
    plt.subplot(1, 2, 1)
    plt.imshow(test_instance)
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.title('LIME Explanation')

    plt.show()

    print("DendCaps Done")

if __name__ == "__main__":
    #initialise()
    DendCaps()